In [1]:
import re, math
import operator
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import StratifiedShuffleSplit
from random import randint
import ipynb.fs.defs.PeopleInfo as peopleInfo
from nltk.corpus import stopwords
from nltk import TweetTokenizer
from nltk import PorterStemmer
# import ipynb.fs.defs.TweetTextHandler as tweetTextHandler
import ipynb.fs.defs.FilterMethods as filterMethods
import sys, os
sys.path.append('../2_feature')
import ipynb.fs.defs.GetFeatures as getFeatures
global stop_words
stop_words = stopwords.words('english')
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from IPython.display import display, Image
plotly.tools.set_credentials_file(username='Adeline', api_key='Z5eltNtBQXqvI05ZFQtz')
# import plotly.offline as offline
# offline.init_notebook_mode(connected=True)

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def replace_by_symbols(txt):
#     txt = re.sub(r"https\S+", 'URL', txt)
#     txt = re.sub(r"http\S+", 'URL', txt)
#     txt = re.sub(r"pic.twitter.com\S+", 'URL', txt)
#     txt = re.sub(r"twitter.com/\S+", 'URL', txt)
#     txt = re.sub(r"\S+/\S+", 'URL', txt)
#     txt = re.sub(r"@\S+", 'USERMENTION', txt)
#     txt = re.sub(r"#\S+", 'HASHTAG', txt)
    txt = re.sub(r"https\S+", '', txt)
    txt = re.sub(r"http\S+", '', txt)
    txt = re.sub(r"pic.twitter.com\S+", '', txt)
    txt = re.sub(r"twitter.com/\S+", '', txt)
    txt = re.sub(r"\S+/\S+", '', txt)
    txt = re.sub(r"@\S+", '', txt)
    txt = re.sub(r"#\S+", '', txt)
    txt = re.sub(r"idk", 'i do not know', txt)   # idk: i don't know
    txt = re.sub(r"tho", 'though', txt)   # tho
    txt = re.sub(r"i\'m", 'i am', txt)
    txt = re.sub(r"you\'re", 'you are', txt)
    txt = re.sub(r"he\'s", 'he is', txt)
    txt = re.sub(r"she\'s", 'she is', txt)
    txt = re.sub(r"it\'s", 'it is', txt)
    txt = re.sub(r"we\'re", 'we are', txt)
    txt = re.sub(r"they\'re", 'they are', txt)
    txt = re.sub(r"isn\'t", 'is not', txt)
    txt = re.sub(r"don\'t", 'do not', txt)
    txt = re.sub(r"doesn\'t", 'does not', txt)
    txt = re.sub(r"didn\'t", 'did not', txt)
    txt = re.sub(r"wasn\'t", 'was not', txt)
    txt = re.sub(r"weren\'t", 'were not', txt)
    txt = re.sub(r"can\'t", 'can not', txt)
    txt = re.sub(r"couldn\'t", 'could not', txt)
    txt = re.sub(r"wouldn\'t", 'would not', txt)
#     txt = re.sub(r"\S+\'ll", '{0} will'.format(txt[0:-3]), txt)
#     txt = re.sub(r"\S+\'ve", '{0} have'.format(txt[0:-3]), txt)
    return txt

In [3]:
patients = dict()
ordinarys = dict()
with open('../0_dataset/patient_ids') as r:
    for patient in r.readlines()[:100]:
        patient = patient.strip()
        patients[patient] = peopleInfo.Patient(patient)
with open('../0_dataset/ordinary_ids') as r:
    for ordinary in r.readlines()[:100]:
        ordinary = ordinary.strip()
        ordinarys[ordinary] = peopleInfo.Ordinary(ordinary)

In [6]:
patients = filterMethods.filter_user_by_tweet_number(patients)
ordinarys = filterMethods.filter_user_by_tweet_number(ordinarys)

Remove users:[]
Remove users:[]


In [7]:
base_texts = []
group_texts = []

for key in patients.keys():
    group_texts.append(replace_by_symbols('\n'.join(patients[key].getText())))

for key in ordinarys.keys():
    base_texts.append(replace_by_symbols('\n'.join(ordinarys[key].getText())))

corpus = base_texts + group_texts

In [12]:
def create_vectorizer():
    vectorizer = TfidfVectorizer(stop_words="english", token_pattern='(?u)\\b[a-zA-Z]\\w{2,}\\b', ngram_range = (1,1), min_df=1)
    return vectorizer

In [38]:
def display_scores(vectorizer, tfidf_result):
    # http://stackoverflow.com/questions/16078015/
    scores = zip(vectorizer.get_feature_names(),
                 np.asarray(tfidf_result.sum(axis=0)).ravel())
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    words = []
    scores = []
    for item in sorted_scores[:100]:
        print("{0:30}\tScore: {1}".format(item[0], item[1]))
#     for item in sorted_scores[:100]:
#         words.append(item[0])
#         scores.append(item[1])
#     return words, scores

# TF-IDF (Mix)

In [39]:
vectorizer = create_vectorizer()
tfidf_result = vectorizer.fit_transform(corpus)
# lst1, lst2 = display_scores(vectorizer, tfidf_result)
display_scores(vectorizer, tfidf_result)
# pd.set_option('display.max_columns', None)
# pd.DataFrame(list(zip(lst1,lst2)), columns=['Word','Score'])

just                          	Score: 17.14452918209988
like                          	Score: 15.37073891937705
love                          	Score: 12.7235574780505
lol                           	Score: 9.33724674489984
know                          	Score: 9.17101903136456
people                        	Score: 8.541506205066401
good                          	Score: 8.269798836092946
want                          	Score: 7.9386156745784096
really                        	Score: 7.795278980867316
day                           	Score: 7.2506707341165315
time                          	Score: 7.190305986253592
thank                         	Score: 7.176296748892114
did                           	Score: 6.963546741778938
today                         	Score: 6.419045311679172
need                          	Score: 6.281900618607257
think                         	Score: 6.176412203613765
got                           	Score: 6.069076297639588
life                          	Score: 5.829735714

# TF-IDF (Base)

In [ ]:
vectorizer = create_vectorizer()
tfidf_result = vectorizer.fit_transform(base_texts)
display_scores(vectorizer, tfidf_result)

In [26]:
vectorizer = create_vectorizer()
tfidf_result = vectorizer.fit_transform(group_texts)
display_scores(vectorizer, tfidf_result)

just	Score: 11.840374598781692
like	Score: 10.607918209544875
love	Score: 8.880044430846423
know	Score: 6.1386632056295465
people	Score: 5.629592469063686
good	Score: 5.379822587056073
want	Score: 5.0014382279572915
lol	Score: 4.928931735236844
really	Score: 4.877489629624206
thank	Score: 4.807460065074168
did	Score: 4.625035564589052
time	Score: 4.2031384705488515
day	Score: 4.138166454199505
life	Score: 3.9296632391972786
shit	Score: 3.9187086575962486
think	Score: 3.917392572752902
need	Score: 3.852857772689931
fuck	Score: 3.8224055026734045
got	Score: 3.82235950693848
feel	Score: 3.79873739989057
fucking	Score: 3.436510966122875
going	Score: 3.40276453896224
lmao	Score: 3.289144765521493
today	Score: 3.2481035674521115
right	Score: 3.017475984781202
happy	Score: 2.9953806994163465
make	Score: 2.874211104494928
new	Score: 2.7849080872323224
gonna	Score: 2.7819650033052223
yes	Score: 2.779388133945607
does	Score: 2.7727102769064693
way	Score: 2.5995805076327834
hate	Score: 2.48414619